# Визуализация решающих деревьев

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 

## Данные

Датасет [Affairs](https://www.kaggle.com/clarkchong/fairs-affairs-dataset) про измены в браке

```
Number of observations: 6366
Number of variables: 9
Variable name definitions:


affairs
numeric. How often engaged in extramarital sexual intercourse during the past year? 0 = none, 1 = once, 2 = twice, 3 = 3 times, 7 = 4–10 times, 12 = monthly, 12 = weekly, 12 = daily.

gender
factor indicating gender.

age
numeric variable coding age in years: 17.5 = under 20, 22 = 20–24, 27 = 25–29, 32 = 30–34, 37 = 35–39, 42 = 40–44, 47 = 45–49, 52 = 50–54, 57 = 55 or over.

yearsmarried
numeric variable coding number of years married: 0.125 = 3 months or less, 0.417 = 4–6 months, 0.75 = 6 months–1 year, 1.5 = 1–2 years, 4 = 3–5 years, 7 = 6–8 years, 10 = 9–11 years, 15 = 12 or more years.

children
factor. Are there children in the marriage?

religiousness
numeric variable coding religiousness: 1 = anti, 2 = not at all, 3 = slightly, 4 = somewhat, 5 = very.

education
numeric variable coding level of education: 9 = grade school, 12 = high school graduate, 14 = some college, 16 = college graduate, 17 = some graduate work, 18 = master's degree, 20 = Ph.D., M.D., or other advanced degree.

occupation
numeric variable coding occupation according to Hollingshead classification (reverse numbering).

rating
numeric variable coding self rating of marriage: 1 = very unhappy, 2 = somewhat unhappy, 3 = average, 4 = happier than average, 5 = very happy.



```


In [ ]:
affairs_data = pd.read_csv('https://raw.githubusercontent.com/esolovev/ling2020/main/lectures/Affairs.csv')

## Подготовка данных

In [ ]:
affairs_data = affairs_data.drop(columns=['Unnamed: 0'])

In [ ]:
affairs_data.shape

In [ ]:
affairs_data.head()

Кодируем строковые переменные:

In [ ]:
affairs_data.gender = [0 if value == 'male' else 1 for value in affairs_data.gender]

In [ ]:
affairs_data.children = [0 if value == 'no' else 1 for value in affairs_data.children]

Давайте закодируем affairs в бинарную переменную и позанимаемся классификацией

In [ ]:
affairs_data.affairs.hist()

In [ ]:
X = affairs_data.drop(columns=['affairs'])

In [ ]:
y = [0 if value == 0 else 1 for value in affairs_data.affairs]

## Обучение

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
SEED = 128

In [ ]:
# количество объектов в классах несбалансированное, 0 сильно больше, чем 1
# используем парметр startify, 
# чтобы в обучающей и тестовой выборке баланс классов был примерно одинаковым
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=SEED)

In [ ]:
dtc = DecisionTreeClassifier(random_state=SEED, max_depth=3)

In [ ]:
dtc.fit(X_train, y_train)

### Посмотрим на качество

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = dtc.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

## Визуализация

### Текстовое представление

https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_text.html#sklearn.tree.export_text

In [ ]:
from sklearn.tree import export_text

In [ ]:
X_train.columns

In [ ]:
text_representation = export_text(dtc, feature_names=list(X_train.columns))
print(text_representation)

### plot_tree из sklearn

https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html#sklearn.tree.plot_tree

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(25,20))
tree_plt = plot_tree(
    dtc, 
    feature_names=list(X_train.columns),
    class_names=['loyal', 'cheating'], # названия классов, 0 - loyal, 1 - cheating 
    filled=True # покрасить в соответвии с количеством объектов правильного класса
    )
# левая стрелка - да, правая стрелка - нет

In [ ]:
# если хотим сохранить картинку
# fig.savefig("decistion_tree.png")

### graphiz

In [ ]:
! pip install graphviz

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

In [ ]:
# dot 
dot_data = export_graphviz(
    dtc, 
    feature_names=list(X_train.columns),
    class_names=['loyal', 'cheating'], # названия классов, 0 - loyal, 1 - cheating  
    filled=True # покрасить в соответвии с количеством объектов правильного класса
    )

In [ ]:
# рисуем дерево
graph = graphviz.Source(dot_data, format="png") 
graph

### dtreeviz

Помимо дерева нарисует симпатичные диаграммы и вообще классно выглядит

In [ ]:
! pip install dtreeviz

In [ ]:
from dtreeviz.trees import dtreeviz

In [ ]:
viz = dtreeviz(dtc, 
               X_train, 
               pd.Series(y_train),
                target_name="target",
                feature_names=X_train.columns,
                title="Affairs dataset classification",
                class_names=['loyal', 'cheating'], 
               scale=1.5 # регулируем размер картинки
               )
viz

**Вопросы**:
+ какие признаки оказались важными, а какие бесполезными для модели?
+ 3 самых важных признака
+ что можно сказать про измены и возраст?
+ что можо сказать про измены и длительность брака?
+ какой узел в дереве бесполезный? почему он появился?